We are going to use gpt 2 simple to generate lyrics.
For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) 

# import data

In [ ]:
import pandas as pd
import pickle

**Note** we have done all necessary preprocessing in preprocessing notebook we will just import clean text file

In [ ]:
with open("/content/drive/MyDrive/Machine learning/RNN/Lyrics_Generation/Data/cleaned_text.txt", 'rb') as fp:
  text= pickle.load(fp)

In [ ]:
text[:5]

['thought id end up with sean',
 'but he wasnt a match',
 'wrote some songs about ricky',
 'now i listen and laugh',
 'even almost got married']

#### This are list of strings we need them joined

In [ ]:
corpus=' '.join(text)

In [ ]:
len(corpus)

7950303

In [ ]:
# save corpus 
with open("/content/drive/MyDrive/Machine learning/RNN/Lyrics_Generation/Data/final_text_corpus.txt", 'wb') as fp:
  pickle.dump(corpus, fp)

# gpt2 stands for ***Generative Pretrained Transformer 2*** :-
* Generative means model was trained to predict or generate the next token in a sequence. OpenAI built this language model using Transformers architecture.
* To learn more about this model visit Jay Alammar's blog [here](https://jalammar.github.io/illustrated-gpt2/)

In [ ]:
# install gpt 2
!pip3 install gpt-2-simple

  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.2-cp37-none-any.whl size=23621 sha256=481e8bbeb9812e98753e9e9d01181b7f753d3700c4c15cc71c7aba95815fd085
  Stored in directory: /root/.cache/pip/wheels/68/1d/15/c87a4302a6c7273ce1b4f282bec3c6877fb2f521535d87d30f
Successfully built gpt-2-simple


**note**- we need tensorflows contib module to work gpt2simple so we will need to select tensorflow version 1 beause they excluded that from version 2

In [ ]:
%tensorflow_version 1.x
import gpt_2_simple as gpt2
from datetime import datetime

## Downloading GPT-2

If we're retraining a model on new text, we need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).


In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 461Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.86Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 608Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [01:49, 4.54Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 412Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.92Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 4.12Mit/s]


The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model out of Colaboratory, is to route it through Google Drive first.

In [10]:
gpt2.mount_gdrive()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_name='/content/drive/MyDrive/Machine learning/RNN/Lyrics_Generation/Data/final_text_corpus.txt'

## Finetune GPT2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

some options of gpt
*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

# Training model 1 `GPT2-124M`

In [ ]:
sess= gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from = 'fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:07<00:00,  7.85s/it]


dataset has 1807805 tokens
Training...
[10 | 27.47] loss=3.02 avg=3.02
[20 | 48.96] loss=2.97 avg=3.00
[30 | 70.62] loss=2.98 avg=2.99
[40 | 92.50] loss=2.69 avg=2.91
[50 | 114.54] loss=2.85 avg=2.90
[60 | 136.79] loss=3.26 avg=2.96
[70 | 159.19] loss=3.01 avg=2.97
[80 | 181.77] loss=3.16 avg=2.99
[90 | 204.46] loss=3.16 avg=3.01
[100 | 227.28] loss=3.24 avg=3.04
[110 | 250.25] loss=2.60 avg=2.99
[120 | 273.34] loss=2.81 avg=2.98
[130 | 296.45] loss=2.61 avg=2.95
[140 | 319.54] loss=2.75 avg=2.93
[150 | 342.62] loss=3.40 avg=2.97
[160 | 365.66] loss=2.85 avg=2.96
[170 | 388.74] loss=3.00 avg=2.96
[180 | 411.79] loss=3.19 avg=2.98
[190 | 434.82] loss=2.79 avg=2.97
[200 | 457.91] loss=3.49 avg=2.99
======== SAMPLE 1 ========
 with no chance to do good so let it go let it go let it go let it go let it go let it go let it go let it go let it go let it go let it go let it go let it go let it go let it go let it go let it go let it go let it go let it go let it go let it go let it go let it 

#### saving model checkpoints to drive

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

Load trained model checkpoints


In [1]:
%tensorflow_version 1.x
import gpt_2_simple as gpt2
from datetime import datetime

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


# Training Model 2 `GPT2-355m`

In [2]:
file_name='/content/drive/MyDrive/Machine learning/RNN/Lyrics_Generation/Data/final_text_corpus.txt'

In [15]:
gpt2.download_gpt2(model_name='355M')

Fetching checkpoint: 1.05Mit [00:00, 690Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.84Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 355Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [03:35, 6.59Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 275Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.33Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 4.26Mit/s]


In [3]:
sess= gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              steps=1000,
              model_name='355M',
              restore_from='fresh',
              run_name='run2',
              print_every=10,
              sample_every=200,
              save_every=500)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:08<00:00,  8.22s/it]


dataset has 1807805 tokens
Training...
[10 | 23.28] loss=3.61 avg=3.61
[20 | 39.51] loss=2.20 avg=2.90
[30 | 56.09] loss=2.64 avg=2.81
[40 | 72.53] loss=2.58 avg=2.75
[50 | 88.74] loss=2.88 avg=2.78
[60 | 104.94] loss=2.53 avg=2.74
[70 | 121.22] loss=2.24 avg=2.66
[80 | 137.61] loss=2.44 avg=2.63
[90 | 153.97] loss=3.34 avg=2.72
[100 | 170.20] loss=1.97 avg=2.64
[110 | 186.43] loss=2.42 avg=2.62
[120 | 202.76] loss=3.94 avg=2.73
[130 | 219.09] loss=2.49 avg=2.71
[140 | 235.44] loss=2.41 avg=2.69
[150 | 251.77] loss=3.25 avg=2.73
[160 | 268.08] loss=2.36 avg=2.71
[170 | 284.36] loss=2.47 avg=2.69
[180 | 300.67] loss=2.68 avg=2.69
[190 | 316.96] loss=4.28 avg=2.78
[200 | 333.26] loss=3.26 avg=2.81
======== SAMPLE 1 ========
 a the first thing i thought was oh aint that why i got so high so high and nothin bout you im here to give this place a try i love the vibe the way the girls run when i see you i know you you dont hate me i know that you like me and i know i know that you want me you

#### save checkpoints to drive

In [4]:
gpt2.copy_checkpoint_to_gdrive(run_name='run2')